In [1]:
# Import libraries
import numpy as np
import geopandas as gpd
import momepy
import networkx as nx
# import pandas as pd
# import shapely
# import shapely.geometry as sg
# import matplotlib
# import matplotlib.pyplot as plt
# %matplotlib inline

from lmzintgraf_gp_pref_elicit import dataset, gaussian_process, acquisition_function
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_ccs as utils_ccs
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_data as utils_data
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_experiment as utils_experiment
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_parameters as utils_parameters
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_user as utils_user

In [93]:
map = gpd.read_file("Sidewalk_width_crossings_small.geojson") #Read in the map with radius 250m and ~1000 nodes

# Objectives
objective1 = map['length']
objective2 = map['crossing']
objective3 = map['obstacle_free_width']

objectives = ('length', 'crossing')

In [125]:
# Create a NetworkX graph from the map
G = momepy.gdf_to_nx(map, approach='primal')
nodes = G.nodes
print(len(nodes))
edges = G.edges
print(len(max(nx.connected_components(G), key=len)))

1006
1006


/usr/local/anaconda3/envs/test/lib/python3.10/site-packages/momepy/utils.py:252: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_network[length] = gdf_network.geometry.length


In [129]:
print(nodes)

[(120548.6120283842, 486088.19577846595), (120558.58272730978, 486088.5913559315), (120562.22249291666, 486088.4604878346), (120554.77791147187, 486105.081631085), (120558.22483637692, 486103.0489683649), (120561.12010043039, 486102.03679359506), (120549.11715177551, 486040.41438763676), (120558.13567293479, 486044.5374951657), (120567.28178312563, 486048.45472129877), (120570.33350867042, 486049.6733418503), (120565.93040988805, 485974.75610421225), (120556.76794233406, 485970.8238889219), (120555.18193758334, 485970.13570051384), (120575.18243990708, 485978.41728733847), (120576.81484759072, 485978.27265655366), (120579.49075192933, 486053.578854089), (120588.64528634987, 486057.48983525735), (120597.74806581473, 486061.48576054414), (120601.8539006359, 486064.20726793807), (120580.87871553846, 485980.01317777124), (120585.59159026924, 485983.13872890506), (120629.68497798537, 486002.06502930616), (120620.5102806444, 485998.1327372588), (120611.32707923822, 485994.21736591624), (1206

In [130]:
#Pick random ones or pick manually that make sense - to experiment
S = (120558.58272730978, 486088.5913559315)
# T = (120798.0893320718, 486128.7633437495)
T = (120683.06067979027, 485777.31229122455)

In [131]:
# Initialise the Gaussian process for 2 objectives
gp = gaussian_process.GPPairwise(num_objectives=2, std_noise=0.01, kernel_width=0.15,prior_mean_type='zero', seed=None)

In [132]:
P = [] #Pareto set
p = [] #paths computed by Dijkstra's algorithm
val_vector_p = [] #value vectors w.r.t. p, i.e., v^{p_1}, v^{p_2}

# Path initialisation
for i in objectives:
    p = nx.shortest_path(G, source=S, target=T, weight=i, method='dijkstra') #Dijkstra's algorithm
    P.append(p)

    val_obj1 = nx.path_weight(G, path=p, weight='length') #Returns total cost associated with the path and weight. In other words, it returns the value of the path.
    val_obj2 = nx.path_weight(G, path=p, weight='crossing')
    val_vector_p.append(np.array([val_obj1, val_obj2]))

In [133]:
val_vector_p

[array([458.83,   6.  ]), array([743.88,   3.  ])]

In [134]:
C = [min(val_vector_p[0][0], val_vector_p[1][0]), min(val_vector_p[0][1], val_vector_p[1][1])] #Candidate Targets, i.e., the most optimistic points
C = [np.array(C)]
C

[array([458.83,   3.  ])]

In [135]:
# User ranking: Compare paths in P
user_preference = utils_user.UserPreference(num_objectives=2, std_noise=0.1)

In [136]:
add_noise = True
ground_utility = user_preference.get_preference(val_vector_p, add_noise=add_noise) #This is the ground-truth utility, i.e., the true utility
print(ground_utility)

[2316.33774202 2316.43835496]


/Users/alisatodorova/Desktop/Thesis/Interactive-MOPPA/lmzintgraf_gp_pref_elicit/gp_utilities/utils_user.py:254: RuntimeWarning: overflow encountered in exp
  y += 1. / (1 + np.exp(- x * (a-i) + (b+i)))


In [137]:
# Add the comparisons to GP
comparisons = dataset.DatasetPairwise(num_objectives=2)

comparisons.add_single_comparison(val_vector_p[np.argmax(ground_utility)], val_vector_p[np.argmin(ground_utility)]) #This way we are performing user ranking of their preferences
print(comparisons.datapoints)
gp.update(comparisons)

[[743.88   3.  ]
 [458.83   6.  ]]


In [138]:
# Find the path the user likes best and has the maximum a posteriori (MAP) estimate
u_v, _ = gp.get_predictive_params(val_vector_p, True) #The maximum a posteriori (MAP) estimate is the mean from gaussian_process.get_predictive_params()
print(u_v)

[-0.02560545  0.02560545]


In [139]:
p_star_index = np.argmax(u_v)
p_star_index

1

In [140]:
p_star = P[p_star_index]
p_star

[(120558.58272730978, 486088.5913559315),
 (120562.22249291666, 486088.4604878346),
 (120597.87366172913, 486118.3885286597),
 (120603.94807229919, 486112.8244804511),
 (120610.1211584085, 486110.4734641692),
 (120615.236697444, 486109.4981894336),
 (120622.18953041689, 486108.8516084536),
 (120627.6055477486, 486108.80491815176),
 (120637.3711853829, 486107.4803808584),
 (120638.32762784461, 486107.6578267999),
 (120644.82334925648, 486107.55756407854),
 (120646.58367929762, 486107.4539825946),
 (120653.63177241654, 486106.9806947992),
 (120658.16234661153, 486106.63926980837),
 (120664.93105615664, 486106.10542439454),
 (120671.8192466126, 486105.5397502568),
 (120678.46251254133, 486104.92881432245),
 (120683.05468505199, 486104.3662392544),
 (120689.70658339545, 486103.4113373837),
 (120697.63022968677, 486102.02209520293),
 (120704.74846380341, 486100.38072378177),
 (120714.24315622302, 486097.30355140124),
 (120715.2451666701, 486096.9044174595),
 (120723.45147424788, 486093.3137

In [141]:
# input_domain = utils_ccs.get_pcs_grid(ccs_size=10, num_objectives=2, eucledian_dist=0.005, min_size=10, seed=123)
# input_domain = np.array(input_domain)
# #Set eucledian_dist smaller to run the code faster
# # Setting ccs_size and min_size to the same value is probably also a good idea.
# # print(input_domain)

In [142]:
input_domain = np.array(C) #set of candidate targets
print(input_domain)

[[458.83   3.  ]]


In [143]:
# Initialise the acquisition function
acq_fun = acquisition_function.DiscreteAcquirer(input_domain=input_domain, query_type='ranking', seed=123, acquisition_type='expected improvement')

In [144]:
# TODO: The next code cells are in a while-loop
# while C:
#     expected_improvement = acquisition_function.get_expected_improvement(input_domain, gp, acq_fun.history)
#     t_index = np.argmax(expected_improvement)
#     t = C[t_index]
#     C.remove(t)
#
# # t_index
# print(t)

In [145]:
expected_improvement = acquisition_function.get_expected_improvement(input_domain, gp, acq_fun.history)
print(expected_improvement)

[0.25035423]


In [146]:
t_index = np.argmax(expected_improvement)
t_index

0

In [147]:
t = input_domain[t_index]
t

array([458.83,   3.  ])

In [148]:
# Remove t from C
# C.remove(t)
# C

In [154]:
v_n = {}
threshold=1e-8
max_iter=1000
obj = 'length'

for n in G:
    v_n[n] = np.inf
    if n == T:
        v_n[n] = 0

for i in range(max_iter): #or until convergence
    v_n_copy = v_n.copy()

    print("iteration:", i)
    for e in G.edges(data=True):
        n1, n2 = e[0], e[1]
        cost = e[2][obj]
        result1 = min(1 + v_n_copy[n2], v_n_copy[n1])
        result2 = min(1 + v_n_copy[n1], v_n_copy[n2])
        v_n[n1] = result1
        v_n[n2] = result2

        # if n1 == T or n2==T:
        #     # print(result1, result2)
        #     print(v_n[n1], v_n[n2])

        # for n_next in G[n]:
        #     for key in G:
        #         # cost = key
        #         # result = np.min(cost + n_next)
        #         result = np.min(key + n_next)
        #         max_value = max(max_value, result)
        #     v_n[n] = max_value
        #
    # converged = all(n in v_n_copy and abs(v_n[n] - v_n_copy[n]) < threshold for n in v_n)
    # if converged:
    #     break

        if v_n[n1] != np.inf:
            print(n1,v_n[n1])
        if v_n[n2] != np.inf:
            print(n2,v_n[n2])

print(v_n)

# import single_vi_iter
# lower_bounds = single_vi_iter.single_value_iter(G)
lower_bounds = v_n


iteration: 0
(120676.90341400128, 485770.02812297666) 1
(120683.06067979027, 485777.31229122455) 0
(120683.06067979027, 485777.31229122455) 0
(120689.21658962962, 485784.5975902885) 1
iteration: 1
(120655.58287503588, 485743.69455813325) 2
(120676.90341400128, 485770.02812297666) 1
(120655.58287503588, 485743.69455813325) 2
(120676.90341400128, 485770.02812297666) 1
(120676.90341400128, 485770.02812297666) 1
(120683.06067979027, 485777.31229122455) 0
(120683.06067979027, 485777.31229122455) 0
(120689.21658962962, 485784.5975902885) 1
iteration: 2
(120655.58287503588, 485743.69455813325) 2
(120656.48350678412, 485734.42161055485) 3
(120655.58287503588, 485743.69455813325) 2
(120676.90341400128, 485770.02812297666) 1
(120655.58287503588, 485743.69455813325) 2
(120676.90341400128, 485770.02812297666) 1
(120676.90341400128, 485770.02812297666) 1
(120683.06067979027, 485777.31229122455) 0
(120683.06067979027, 485777.31229122455) 0
(120689.21658962962, 485784.5975902885) 1
iteration: 3
(1206

In [116]:
i = 0  # track iterations of the algorithm
max_iter=None

stack = [(S, 0, [S])]  # (starting node, cost, path), where cost is from previous_state to S, path is from S to current_state (i.e., S)

while stack:
    current_node, cost, path = stack.pop()  # cost=total cost up to the current_node

    if current_node == T:
        print(f"Path {path} with cost {cost}")

    for next_node in G[current_node]:
        for key in G:
            total_cost = key + next_node
            new_path = path + [next_node] # Add the neighbor to the path
        # print(new_path)
        stack.append((next_node, total_cost, new_path))
        # print(stack)
        stack.sort(key=lambda x: lower_bounds[x[0]],
                       reverse=True)  # Sorts in descending order w.r.t. the lower bound

    i += 1
    if max_iter is not None and i >= max_iter:
        print("The algorithm has reached the given maximum iterations, but has found no solution.")
        break


print("The algorithm has terminated, but no solution was found.")
#TODO: This keeps on compiling for max_iter>10000, but for max_iter<10000 it terminates with no solutions.

KeyboardInterrupt: 

In [73]:
# Inner-loop approach
import dfs_lower
p_s = dfs_lower.dfs_lower(G, S, t, lower_bounds)
print(p_s)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
#TODO: Line 15 of pseudocode is unclear to me how it should be in code.
# If v^p improves in the target region
# because you've identified a new value vector on the PCS. If you stop once the utility no longer improves, I think this can result in stopping prematurely. Specifically, imagine you have a current partial Pareto front of (10,0) and (0,10) the user model u((10,0)) is the current best. The target vector is (10,10)  and when you run DFS, you get one of the possible vectors in the target region. You get (1,9) out of the the call to DFS, and the u((1,9)) < u((10,0)) even after querying the user about it. Now here, you shouldn't stop, because the true best - (7,3) for example, is still possible to find.
#you're just not going to improve on that with a newly found vector
#So improving on the acquisition function by identifying a new point is impossible as
# 1) you were searching at an optimistic estimate (target), so the actually found value will be worse than the target
# 2) finding new points, and querying the user reduces uncertainty

In [ ]:
P = P.append(p_s)

In [ ]:
#TODO: Can't test because of p_s (see above)
#Compare p^s to p^∗ and add comparison to the GP ▷User ranking, i.e., is the new path preferred to the current, maximum one?

val_vector_new_paths = [] #value vectors of paths p^s and p^∗

for i in objectives:
    val_p_s = nx.path_weight(G, path=p_s, weight=i)
    val_p_star = nx.path_weight(G, path=p_star, weight=i)

    val_vector_new_paths.append(np.array([val_p_s, val_p_star]))


In [ ]:
ranking_new_paths = user_preference.get_preference(val_vector_new_paths, add_noise=add_noise)
print(ranking_new_paths)

In [ ]:
# Add the comparisons to GP
comparisons.add_single_comparison(val_vector_new_paths[np.argmax(ranking_new_paths)], val_vector_new_paths[
    np.argmin(ranking_new_paths)])  #This way we are performing user ranking of their preferences
print(comparisons.datapoints)
gp.update(comparisons)

In [ ]:
u_v_p_s, _ = gp.get_predictive_params(val_p_s, True)  #The maximum a posteriori (MAP) estimate is the mean from gaussian_process.get_predictive_params()
u_v_p_star, _ = gp.get_predictive_params(val_p_star, True)

In [ ]:
# if u(v^{p^s}) > u(v^{p^*}) then
if u_v_p_s > u_v_p_star:
    #p^∗ ← p^s
    p_star = p_s
# end if

In [ ]:
# Compute new candidate targets based on v^{p^s} and add to C
C = [min(val_p_s[0][0], val_p_s[1][0]), min(val_p_s[0][1], val_p_s[1][1])]
C

In [ ]:
# end if
# end while
# return p^∗, v^{p^*}
